In [1]:
def hex_to_binary(hex_str,bit_length):
    """Convert hex string to binary string."""
    return str(bin(int(hex_str, 16))[2:].zfill(bit_length)) if bit_length else str(bin(int(hex_str, 16))[2:])

def binary_to_int(binary_str):
    return str(int(binary_str, 2))

def hex_to_decimal(hex_str):
    # 원래 16진수 문자열의 길이를 계산합니다.
    original_length = len(hex_str)
    # 16진수 문자열을 10진수 정수로 변환합니다.
    decimal_number = str(int(hex_str, 16))
    # 10진수 문자열의 길이를 계산합니다.
    decimal_length = len(decimal_number)
    # 원래 길이와 현재 길이를 비교하여 부족한 만큼 앞에 0을 추가합니다.
    if decimal_length < original_length:
        decimal_number = decimal_number.zfill(original_length)
    return decimal_number

def insert_dot_at_second_position(s):
    return s[:-6] + '.' + s[-6:]

def parse_hexstring(hexstring):
    """ Parse the hexstring to extract the data fields. """
    data = {}
    data['DATA_LENGTH'] = len(hexstring)
    data['SOF'] = hexstring[0:2]
    data['MSG_ID'] = hexstring[2:4]
    data['Sequence'] = hexstring[6:8] + hexstring[4:6]
    data['PAYLOAD_LENGTH'] = hexstring[8:10]
    data['BUOY_ID'] = hexstring[24:26] + hexstring[22:24] + hexstring[20:22] + hexstring[18:20] + hexstring[16:18] + hexstring[14:16] + hexstring[12:14] + hexstring[10:12]
    data['BUOY_STATUS_HEX'] = hexstring[28:30] + hexstring[26:28]
    data['BUOY_STATUS_BINARY'] = hex_to_binary(hexstring[28:30],8)  + hex_to_binary(hexstring[26:28], 8)
    data['WIRELESS_STATUS'] = data['BUOY_STATUS_BINARY'][0:1]
    data['MAIN_MODULE_STATUS'] = data['BUOY_STATUS_BINARY'][1:2]
    data['PLORA_MODULE_STATUS'] = data['BUOY_STATUS_BINARY'][2:3]
    data['GNSS_STATUS'] = data['BUOY_STATUS_BINARY'][3:4]
    data['POWER'] = binary_to_int(data['BUOY_STATUS_BINARY'][4:7])
    data['LORA_STATUS'] = binary_to_int(data['BUOY_STATUS_BINARY'][7:10])
    data['PLORA_STATUS'] = binary_to_int(data['BUOY_STATUS_BINARY'][10:13])
    data['FISHINGBOAT_GW_STATUS'] = binary_to_int(data['BUOY_STATUS_BINARY'][13:16])
    data['YEAR'] = hex_to_decimal(hexstring[32:34] + hexstring[30])
    data['MONTH'] = hex_to_decimal(hexstring[31])
    data['DHM'] = hex_to_binary(hexstring[36:38],8) + hex_to_binary(hexstring[34:36],8)
    data['DAY'] = binary_to_int(data['DHM'][0:5])
    data['HOUR'] = int(binary_to_int(data['DHM'][5:10])) + int(binary_to_int('1001'))
    data['MIN'] = binary_to_int(data['DHM'][10:16])
    data['SECOND'] = binary_to_int(hex_to_binary(hexstring[38:40],0)[0:6])
    data['GPS_STATUS'] = hex_to_binary(hexstring[40:42],8)
    data['GNSS_STATUS'] = data['GPS_STATUS'][0]
    data['LATITUDE_CONDITION'] = data['GPS_STATUS'][1]
    data['LONGITUDE_CONDITION'] = data['GPS_STATUS'][2]
    data['GPS_MANUFACTURER'] = data['GPS_STATUS'][3:6]
    data['PRELIMINARY_VALUE'] = data['GPS_STATUS'][6:8]
    data['LATITUDE'] = insert_dot_at_second_position(str(int(hex_to_decimal(hexstring[48:50] + hexstring[46:48]))//2) + hex_to_decimal(hexstring[44:46] + hexstring[42:44]))
    data['LONGITUDE'] = insert_dot_at_second_position(str(int(hex_to_decimal(hexstring[56:58] + hexstring[54:56]))//2) + hex_to_decimal(hexstring[52:54] + hexstring[50:52]))

    return data

In [6]:
# parse_hexstring('aa316808186dec20feff1d5788ad06697e486b80104300f01ab6248c62c7fe')
# parse_hexstring('aa316608186dec20feff1d5788ad06697e466b2c10d109f01a821b8c62c9fe')
# parse_hexstring('aa316708186dec20feff1d57886d06697e476b54108e04f01a2a208c6218fe')
parse_hexstring('aa310200184aed1dfeff63ca4000e8877e755350105c0c4a1cfd086463bdfe')

{'DATA_LENGTH': 62,
 'SOF': 'aa',
 'MSG_ID': '31',
 'Sequence': '0002',
 'PAYLOAD_LENGTH': '18',
 'BUOY_ID': '40ca63fffe1ded4a',
 'BUOY_STATUS_HEX': 'e800',
 'BUOY_STATUS_BINARY': '1110100000000000',
 'WIRELESS_STATUS': '1',
 'MAIN_MODULE_STATUS': '1',
 'PLORA_MODULE_STATUS': '1',
 'GNSS_STATUS': '0',
 'POWER': '4',
 'LORA_STATUS': '0',
 'PLORA_STATUS': '0',
 'FISHINGBOAT_GW_STATUS': '0',
 'YEAR': '2024',
 'MONTH': '7',
 'DHM': '0101001101110101',
 'DAY': '10',
 'HOUR': 22,
 'MIN': '53',
 'SECOND': '40',
 'GPS_STATUS': '00010000',
 'LATITUDE_CONDITION': '0',
 'LONGITUDE_CONDITION': '0',
 'GPS_MANUFACTURER': '100',
 'PRELIMINARY_VALUE': '00',
 'LATITUDE': '36.213164',
 'LONGITUDE': '127.222301'}

In [2]:
from paho.mqtt.client import Client
import json
mqtt_client = Client()

MQTT_BROKER = '14.50.159.2'
MQTT_PORT = 1883
# MQTT_USER = 'rcnapp1@ttn'
# MQTT_PASSWORD = 'NNSXS.YCU7AAWPOCH27N6UEVVRZSF33IVIDMQRB6L2AHI.THLGQXBMXSESTGLXICJZQQGWJRMZAEARP7VQPTZI7SEYJHC7GLXA'

# mqtt_client.username_pw_set(MQTT_USER, MQTT_PASSWORD)
mqtt_client.connect(MQTT_BROKER, MQTT_PORT, 60)
MQTT_TOPIC = 'v3/rcnapp1@ttn/devices/105838541867477030805/up'
data = { # 위치 무궁화빌라
    "received_at": "2024-07-10 17:19:00", # test할 때 +9GMT 신경써서
    "frequency": "9552110",
    "hexString": 'aa310200184aed1dfeff63ca4000e8877e755350105c0c4a1cfd086463bdfe',
    "dev_eui": "40ca63fffe1ded4a",
    "dev_addr": "11100009",
    "application_id":"appid123",
    "parsed_string": {
        "BUOY_ID": "40ca63fffe1ded4a",
        "BUOY_STATUS_BINARY": "1110100000000000",
        "BUOY_STATUS_HEX": "e800",
        "DATA_LENGTH": 62,
        "DAY": "10",
        "DHM": "0101001101110101",
        "FISHINGBOAT_GW_STATUS": "0",
        "GNSS_STATUS": "0",
        "GPS_MANUFACTURER": "100",
        "GPS_STATUS": "00010000",
        "HOUR": 22,
        "LATITUDE": "36.354164",
        "LATITUDE_CONDITION": "0",
        "LONGITUDE": "127.370301",
        "LONGITUDE_CONDITION": "0",
        "LORA_STATUS": "0",
        "MAIN_MODULE_STATUS": "1",
        "MIN": "53",
        "MONTH": "7",
        "MSG_ID": "31",
        "PAYLOAD_LENGTH": "18",
        "PLORA_MODULE_STATUS": "1",
        "PLORA_STATUS": "0",
        "POWER": "4",
        "PRELIMINARY_VALUE": "00",
        "SECOND": "40",
        "SOF": "aa",
        "Sequence": "0002",
        "WIRELESS_STATUS": "1",
        "YEAR": "2024"
    }
}
data2 = {
    "received_at": "2024-07-10 17:19:00", # test할 때 +9GMT 신경써서
    "frequency": "9552110",
    "hexString": 'aa310200184aed1dfeff63ca4000e8877e755350105c0c4a1cfd086463bdfe',
    "dev_eui": "40ca63fffe1ded4b",
    "dev_addr": "11100009",
    "application_id":"appid123",
    "parsed_string": {
        "BUOY_ID": "40ca63fffe1ded4b",
        "BUOY_STATUS_BINARY": "1110100000000000",
        "BUOY_STATUS_HEX": "e800",
        "DATA_LENGTH": 62,
        "DAY": "10",
        "DHM": "0101001101110101",
        "FISHINGBOAT_GW_STATUS": "0",
        "GNSS_STATUS": "0",
        "GPS_MANUFACTURER": "100",
        "GPS_STATUS": "00010000",
        "HOUR": 22,
        "LATITUDE": "36.454164",
        "LATITUDE_CONDITION": "0",
        "LONGITUDE": "127.470301",
        "LONGITUDE_CONDITION": "0",
        "LORA_STATUS": "0",
        "MAIN_MODULE_STATUS": "1",
        "MIN": "53",
        "MONTH": "7",
        "MSG_ID": "31",
        "PAYLOAD_LENGTH": "18",
        "PLORA_MODULE_STATUS": "1",
        "PLORA_STATUS": "0",
        "POWER": "4",
        "PRELIMINARY_VALUE": "00",
        "SECOND": "40",
        "SOF": "aa",
        "Sequence": "0002",
        "WIRELESS_STATUS": "1",
        "YEAR": "2024"
    }
}
data3 = {
    "received_at": "2024-07-10 17:24:00", # test할 때 +9GMT 신경써서
    "frequency": "9552110",
    "hexString": 'aa310200184aed1dfeff63ca4000e8877e755350105c0c4a1cfd086463bdfe',
    "dev_eui": "40ca63fffe1ded4c",
    "dev_addr": "11100009",
    "application_id":"appid123",
    "parsed_string": {
        "BUOY_ID": "40ca63fffe1ded4c",
        "BUOY_STATUS_BINARY": "1110100000000000",
        "BUOY_STATUS_HEX": "e800",
        "DATA_LENGTH": 62,
        "DAY": "10",
        "DHM": "0101001101110101",
        "FISHINGBOAT_GW_STATUS": "0",
        "GNSS_STATUS": "0",
        "GPS_MANUFACTURER": "100",
        "GPS_STATUS": "00010000",
        "HOUR": 22,
        "LATITUDE": "36.254164",
        "LATITUDE_CONDITION": "0",
        "LONGITUDE": "127.270301",
        "LONGITUDE_CONDITION": "0",
        "LORA_STATUS": "0",
        "MAIN_MODULE_STATUS": "1",
        "MIN": "53",
        "MONTH": "7",
        "MSG_ID": "31",
        "PAYLOAD_LENGTH": "18",
        "PLORA_MODULE_STATUS": "1",
        "PLORA_STATUS": "0",
        "POWER": "4",
        "PRELIMINARY_VALUE": "00",
        "SECOND": "40",
        "SOF": "aa",
        "Sequence": "0002",
        "WIRELESS_STATUS": "1",
        "YEAR": "2024"
    }
}
mqtt_client.publish(MQTT_TOPIC, json.dumps(data))
mqtt_client.publish(MQTT_TOPIC, json.dumps(data2))
mqtt_client.publish(MQTT_TOPIC, json.dumps(data3))

ModuleNotFoundError: No module named 'paho'